## Calculating PM2.5

In [2]:
import os
import pandas as pd
import numpy as np

# Set the directory path
metadata_dir = r'C:\Users\Akanksha\OneDrive\Desktop\data\Metadata'

def load_and_standardize(filepath):
    if filepath.endswith('.xlsx'):
        df = pd.read_excel(filepath)
    else:
        df = pd.read_csv(filepath)
    
    # Rename inconsistent column names
    rename_dict = {
        "Mass collected on filter": "Mass collected on filter (ug)",
        "Sampled volume": "Sampled volume (m3)"
    }
    df.rename(columns=rename_dict, inplace=True)
    
    return df

combined_df = pd.concat(
    [
        load_and_standardize(os.path.join(metadata_dir, f))
        for f in os.listdir(metadata_dir)
        if f.endswith('.xlsx') or f.endswith('.csv')
    ],
    ignore_index=True
)

# Remove rows with 'Filter Type' == 'FB'
combined_df = combined_df[combined_df["Filter Type"] != "FB"].copy()

# Clean up column names: strip spaces and unify formatting
combined_df.columns = combined_df.columns.str.strip()

# Drop irrelevant columns
columns_to_drop = ["Shipment ID (Date)", "Shipment ID", "Cartridge Number", "Barcode", "Project ID", "Lot ID", "Comments"]
combined_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Convert to numeric safely
combined_df["Sampled volume (m3)"] = pd.to_numeric(combined_df["Sampled volume (m3)"], errors='coerce')
combined_df["Mass collected on filter (ug)"] = pd.to_numeric(combined_df["Mass collected on filter (ug)"], errors='coerce')

# Remove rows with zero or NaN volume before division
combined_df = combined_df[
    (combined_df["Sampled volume (m3)"].notna()) &
    (combined_df["Sampled volume (m3)"] != 0)
].copy()

# Now safely calculate PM2.5
combined_df["PM2.5(ug/m3)"] = combined_df["Mass collected on filter (ug)"] / combined_df["Sampled volume (m3)"]

# Filter only PM2.5 filters
pm25_df = combined_df[combined_df["Filter Type"] == "PM2.5"].copy()
pm25_df.loc[pm25_df["PM2.5(ug/m3)"] <= 0, "PM2.5(ug/m3)"] = np.nan
pm25_df.dropna(subset=["PM2.5(ug/m3)"], inplace=True)

In [3]:
pm25_df.columns

Index(['Filter ID', 'Analysis ID', 'Filter Type', 'Sampling Start Date',
       'Sampling End Date', 'Mass collected on filter (ug)',
       'Sampled volume (m3)', 'PM2.5(ug/m3)'],
      dtype='object')

In [4]:
pm25_df["Filter ID"] = pm25_df["Filter ID"].str.replace(r'\s*-\s*', '-', regex=True).str.strip()

In [5]:
# Add a blank average row to PM2.5 DataFrame
blank_row = {
    'Filter ID': 'AAAA-BLNK-X',
    'Analysis ID': 'AAAA-BLNK-X',
    'Filter Type': 'NF',
    'Sampling Start Date': '0',
    'Sampling End Date': '0',
    'Mass collected on filter (ug)': 1,
    'Sampled volume (m3)': 1,
    'PM2.5(ug/m3)': 1
}
pm25_df = pd.concat([pm25_df, pd.DataFrame([blank_row])], ignore_index=True)

# Display the PM2.5 DataFrame 
display(pm25_df)

pm25_df.to_csv("pm25_df.csv", index=False)

,Filter ID,Analysis ID,Filter Type,Sampling Start Date,Sampling End Date,Mass collected on filter (ug),Sampled volume (m3),PM2.5(ug/m3)
0,USPA-0161-1,USPA-0161,PM2.5,10/14/2022,10/15/2022,56.7,7.344,7.720588
1,USPA-0162-2,USPA-0162,PM2.5,10/17/2022,10/18/2022,52.9,7.272,7.274477
2,USPA-0163-3,USPA-0163,PM2.5,10/20/2022,10/21/2022,36.0,7.272,4.950495
3,USPA-0164-4,USPA-0164,PM2.5,10/23/2022,10/24/2022,40.0,7.272,5.500550
4,USPA-0165-5,USPA-0165,PM2.5,10/26/2022,10/27/2022,44.2,7.272,6.078108
...,...,...,...,...,...,...,...,...
2781,PRFJ-0074-2,PRFJ-0074,PM2.5,2/5/2023,2/13/2023,27.3,7.200,3.791667
2782,PRFJ-0075-3,PRFJ-0075,PM2.5,2/14/2023,2/22/2023,30.7,7.200,4.263889
2783,PRFJ-0076-4,PRFJ-0076,PM2.5,2/23/2023,3/3/2023,39.1,7.200,5.430556
2784,PRFJ-0077-5,PRFJ-0077,PM2.5,3/4/2023,3/12/2023,36.0,6.798,5.295675


### Names of filters in reflectance folder

In [7]:
import os

Reflectance_directory_path = r"C:\Users\Akanksha\OneDrive\Desktop\data\ALL_Reflectance"
path = r"C:\Users\Akanksha\OneDrive\Desktop\data"

filter_ids = []

for filename in os.listdir(Reflectance_directory_path):
    if filename.endswith(".csv"):
        filter_id = filename.split(".")[0]
        filter_ids.append(filter_id)

output_file = os.path.join(path, "list_of_filter_ids_ref.txt")
with open(output_file, "w") as f:
    for fid in filter_ids:
        f.write(fid + "\n")

### Names of filters in transmittance folder

In [9]:
import os

Transmittance_directory_path = r"C:\Users\Akanksha\OneDrive\Desktop\data\ALL_Transmittance"
path = r"C:\Users\Akanksha\OneDrive\Desktop\data"

filter_ids = []

for filename in os.listdir(Transmittance_directory_path):
    if filename.endswith(".csv"):
        filter_id = filename.split(".")[0]
        filter_ids.append(filter_id)

output_file = os.path.join(path, "list_of_filter_ids_tran.txt")
with open(output_file, "w") as f:
    for fid in filter_ids:
        f.write(fid + "\n")

### Python Code to Compare the Two Files

In [11]:
file1 = r"C:\Users\Akanksha\OneDrive\Desktop\data\list_of_filter_ids_ref.txt"
file2 = r"C:\Users\Akanksha\OneDrive\Desktop\data\list_of_filter_ids_tran.txt"

# Read and clean lines
with open(file1, 'r') as f1, open(file2, 'r') as f2:
    set1 = set(line.strip() for line in f1 if line.strip())
    set2 = set(line.strip() for line in f2 if line.strip())

# Differences
only_in_file1 = sorted(set1 - set2)
only_in_file2 = sorted(set2 - set1)
common = sorted(set1 & set2)

# Print summary
print(f"Only in file_tra ({os.path.basename(file1)}): {len(only_in_file1)} items")
print(f"Only in file_ref ({os.path.basename(file2)}): {len(only_in_file2)} items")
print(f"Common items: {len(common)}")

# Save results (optional)
base_dir = os.path.dirname(file1)

with open(os.path.join(base_dir, "only_in_file_tra.txt"), "w") as f:
    f.writelines(line + "\n" for line in only_in_file1)

with open(os.path.join(base_dir, "only_in_file_ref.txt"), "w") as f:
    f.writelines(line + "\n" for line in only_in_file2)

with open(os.path.join(base_dir, "common_filters.txt"), "w") as f:
    f.writelines(line + "\n" for line in common)

Only in file_tra (list_of_filter_ids_ref.txt): 5 items
Only in file_ref (list_of_filter_ids_tran.txt): 4 items
Common items: 2301


### Moved files without PM2.5 data 

In [13]:
import os
import shutil
import pandas as pd

source_dir = r"C:\Users\Akanksha\OneDrive\Desktop\data\ALL_Reflectance"  
target_dir = os.path.join(source_dir, "Files_without_PM25")

log_file_path = "moved_files_ref.txt"

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Get list of valid Filter IDs
valid_filter_ids = set(pm25_df['Filter ID'].tolist())

# Open log file for writing
with open(log_file_path, "w") as log_file:
    for filename in os.listdir(source_dir):
        if filename.endswith(".csv"):
            base_name = filename.split(".")[0]
            if base_name not in valid_filter_ids:
                # Move file
                src_path = os.path.join(source_dir, filename)
                dst_path = os.path.join(target_dir, filename)
                shutil.move(src_path, dst_path)
                
                # Log moved file
                log_file.write(filename + "\n")

In [14]:
import os
import shutil
import pandas as pd

source_dir = r"C:\Users\Akanksha\OneDrive\Desktop\data\ALL_Transmittance"  
target_dir = os.path.join(source_dir, "Files_without_PM25")

log_file_path = "moved_files_tran.txt"

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Get list of valid Filter IDs
valid_filter_ids = set(pm25_df['Filter ID'].tolist())

# Open log file for writing
with open(log_file_path, "w") as log_file:
    for filename in os.listdir(source_dir):
        if filename.endswith(".csv"):
            base_name = filename.split(".")[0]
            if base_name not in valid_filter_ids:
                # Move file
                src_path = os.path.join(source_dir, filename)
                dst_path = os.path.join(target_dir, filename)
                shutil.move(src_path, dst_path)
                
                # Log moved file
                log_file.write(filename + "\n")

### common filters in ref and trans

In [16]:
# Paths to your two input .txt files
file1_path = r"C:\Users\Akanksha\Documents\Untitled Folder\moved_files_ref.txt"  
file2_path = r"C:\Users\Akanksha\Documents\Untitled Folder\moved_files_tran.txt"  

# Function to extract filter IDs from file
def extract_filter_ids(file_path):
    with open(file_path, 'r') as file:
        return set(line.strip().split('.')[0] for line in file if line.strip())

# Extract filter IDs from both files
filters_1 = extract_filter_ids(file1_path)
filters_2 = extract_filter_ids(file2_path)

# Find common filters
common_filters = filters_1.intersection(filters_2)

# Save common filter names to common.txt
with open("common.txt", "w") as f:
    for filter_id in sorted(common_filters):
        f.write(filter_id + "\n")

print("Saved common filter names to common.txt")

Saved common filter names to common.txt


### Cross check

In [18]:
with open("common.txt", "r") as f:
    common_ids = set(line.strip() for line in f if line.strip())

# Get Filter IDs from pm25_df as a set
pm25_ids = set(pm25_df['Filter ID'].astype(str))

# Find matching IDs
matched_ids = common_ids.intersection(pm25_ids)

# Report result
if matched_ids:
    print("Matches found:")
    for fid in sorted(matched_ids):
        print(fid)
else:
    print("No matches found between common.txt and pm25_df['Filter ID']")


No matches found between common.txt and pm25_df['Filter ID']
